In [1]:
import sys
sys.path.insert(0,"../../")

from library import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import *
from bs4.element import PageElement


# 2021 Playoffs

## Playoff Series

In [2]:
'https://www.basketball-reference.com/playoffs/NBA_2021.html'

'https://www.basketball-reference.com/playoffs/NBA_2021.html'

In [3]:
source = get_url(f'https://www.basketball-reference.com/playoffs/NBA_2021.html')
source = BeautifulSoup(strip_html_comment(source))
selector = '#all_playoffs'
divs = source.select(selector)
divs

[<table class="suppress_all sortable stats_table" data-cols-to-freeze="0" id="all_playoffs">
 <caption>Playoff Series Table</caption>
 <colgroup></colgroup>
 <thead> </thead>
 <tbody>
 <tr>
 <td><span class="tooltip opener" data-id="s904"><strong>Finals</strong></span></td>
 <td><a href="/teams/MIL/2021.html">Milwaukee Bucks</a> 
 over 
 <a href="/teams/PHO/2021.html">Phoenix Suns</a>
  (4-2)
 </td>
 <td><a href="/playoffs/2021-nba-finals-bucks-vs-suns.html">Series Stats</a></td>
 </tr>
 <tr class="toggleable" id="s904">
 <td colspan="3"><div>
 <table>
 <tr>
 <td><a href="/boxscores/202107060PHO.html">Game 1</a></td>
 <td>Tue, July 6</td>
 <td>Milwaukee Bucks</td>
 <td>105</td><td>@ Phoenix Suns</td>
 <td>118</td></tr>
 <tr>
 <td><a href="/boxscores/202107080PHO.html">Game 2</a></td>
 <td>Thu, July 8</td>
 <td>Milwaukee Bucks</td>
 <td>108</td><td>@ Phoenix Suns</td>
 <td>118</td></tr>
 <tr>
 <td><a href="/boxscores/202107110MIL.html">Game 3</a></td>
 <td>Sun, July 11</td>
 <td>Phoenix

## Playoffs Summary

In [97]:
def get_playoffs_summary(year=2021):
    source = get_url(f'https://www.basketball-reference.com/playoffs/NBA_{year}.html')
    source = BeautifulSoup(strip_html_comment(source))
    selector = '#all_playoffs > tbody > tr:not(.toggleable,.thead)'
    divs = source.select(selector)
    df = []
    for div in divs: 
        po_round = div.select_one(".tooltip.opener")
        score = div.select("td")[1]
        W,L,series = div.select("td > a")[:3]
        d = {
            'round' : po_round.text,
            'W'     : W.text,
            'L'     : L.text,
            **re.search(r"((?P<W_score>\d)\-(?P<L_score>\d))",score.text).groupdict(),
            'W_url': f"{W['href']}",
            'L_url': f"{L['href']}",
            'series_stats': f"{series['href']}"
        }
        df.append(d)
    df = pd.DataFrame(df)
    return df

get_playoffs_summary(2021)

,round,W,L,W_score,L_score,W_url,L_url,series_stats
0,Finals,Milwaukee Bucks,Phoenix Suns,4,2,/teams/MIL/2021.html,/teams/PHO/2021.html,/playoffs/2021-nba-finals-bucks-vs-suns.html
1,Eastern Conference Finals,Milwaukee Bucks,Atlanta Hawks,4,2,/teams/MIL/2021.html,/teams/ATL/2021.html,/playoffs/2021-nba-eastern-conference-finals-h...
2,Western Conference Finals,Phoenix Suns,Los Angeles Clippers,4,2,/teams/PHO/2021.html,/teams/LAC/2021.html,/playoffs/2021-nba-western-conference-finals-c...
3,Eastern Conference Semifinals,Atlanta Hawks,Philadelphia 76ers,4,3,/teams/ATL/2021.html,/teams/PHI/2021.html,/playoffs/2021-nba-eastern-conference-semifina...
4,Eastern Conference Semifinals,Milwaukee Bucks,Brooklyn Nets,4,3,/teams/MIL/2021.html,/teams/BRK/2021.html,/playoffs/2021-nba-eastern-conference-semifina...
5,Western Conference Semifinals,Los Angeles Clippers,Utah Jazz,4,2,/teams/LAC/2021.html,/teams/UTA/2021.html,/playoffs/2021-nba-western-conference-semifina...
6,Western Conference Semifinals,Phoenix Suns,Denver Nuggets,4,0,/teams/PHO/2021.html,/teams/DEN/2021.html,/playoffs/2021-nba-western-conference-semifina...
7,Eastern Conference First Round,Atlanta Hawks,New York Knicks,4,1,/teams/ATL/2021.html,/teams/NYK/2021.html,/playoffs/2021-nba-eastern-conference-first-ro...
8,Eastern Conference First Round,Brooklyn Nets,Boston Celtics,4,1,/teams/BRK/2021.html,/teams/BOS/2021.html,/playoffs/2021-nba-eastern-conference-first-ro...
9,Eastern Conference First Round,Milwaukee Bucks,Miami Heat,4,0,/teams/MIL/2021.html,/teams/MIA/2021.html,/playoffs/2021-nba-eastern-conference-first-ro...


In [112]:
def get_playoffs_schedule(year=2021):
    source = get_url(f'https://www.basketball-reference.com/playoffs/NBA_{year}.html')
    source = BeautifulSoup(strip_html_comment(source))
    selector = '#all_playoffs > tbody > tr:not(.thead)'
    divs = source.select(selector)

    df = []
    for div_series_summary,div_series_games in zip(divs[0::2],divs[1::2]):
        po_round = div_series_summary.select_one(".tooltip.opener").text
        for div_game in div_series_games.select('tr'):
            game,date,v,pts_v,h,pts_h = div_game.select('td')
            box_scores = re.findall(r"/boxscores/(.+).html",game.a['href'])[0] if game.a else ""
            d = {
                'round':po_round,
                'game' : game.text,
                'date' : date.text,
                'visitor': v.text,
                'home': re.sub("@ ","",h.text),
                'pts_v': pts_v.text,
                'pts_h': pts_h.text,
                'box_scores': box_scores
                }
            df.append(d)
    df = pd.DataFrame(df)
    return df

get_playoffs_schedule(year=2021)


,round,game,date,visitor,home,pts_v,pts_h,box_scores
0,Finals,Game 1,"Tue, July 6",Milwaukee Bucks,Phoenix Suns,105,118,202107060PHO
1,Finals,Game 2,"Thu, July 8",Milwaukee Bucks,Phoenix Suns,108,118,202107080PHO
2,Finals,Game 3,"Sun, July 11",Phoenix Suns,Milwaukee Bucks,100,120,202107110MIL
3,Finals,Game 4,"Wed, July 14",Phoenix Suns,Milwaukee Bucks,103,109,202107140MIL
4,Finals,Game 5,"Sat, July 17",Milwaukee Bucks,Phoenix Suns,123,119,202107170PHO
...,...,...,...,...,...,...,...,...
80,Western Conference First Round,Game 1,"Sun, May 23",Memphis Grizzlies,Utah Jazz,112,109,202105230UTA
81,Western Conference First Round,Game 2,"Wed, May 26",Memphis Grizzlies,Utah Jazz,129,141,202105260UTA
82,Western Conference First Round,Game 3,"Sat, May 29",Utah Jazz,Memphis Grizzlies,121,111,202105290MEM
83,Western Conference First Round,Game 4,"Mon, May 31",Utah Jazz,Memphis Grizzlies,120,113,202105310MEM


## Playoffs Stats

In [199]:
def get_playoffs_stats(year = 2021):
    source = get_url(f'https://www.basketball-reference.com/playoffs/NBA_{year}.html')
    source = BeautifulSoup(strip_html_comment(source))

    def get_data_stats(source,selector,select='td'):
        divs = source.select(selector)
        data_stats = pd.DataFrame([{d['data-stat']:d.text for d in div_team.select(select)} for div_team in divs])
        return data_stats

    playoff_stats = {
        'per-game-team': get_data_stats(source,'#per_game-team > tbody > tr'),
        'per-game-opponent': get_data_stats(source,'#per_game-opponent> tbody > tr'),
        'totals-team': get_data_stats(source,'#totals-team > tbody > tr'),
        'totals-opponent': get_data_stats(source,'#totals-opponent > tbody > tr'),
        'per-100-poss-team': get_data_stats(source,'#per_poss-team > tbody > tr'),
        'per-100-poss-opponent': get_data_stats(source,'#per_poss-team > tbody > tr'),
        'advanced-team': get_data_stats(source,'#advanced-team > tbody > tr'),
        'shooting-team': get_data_stats(source,'#shooting-team > tbody > tr'),
        'shooting-opponent': get_data_stats(source,'#shooting-opponent > tbody > tr'),
    }
    return playoff_stats
d = get_playoffs_stats()

In [200]:
from IPython.display import *

def iter_display(d,sleep=1):
    for k,v in d.items():
        clear_output()
        print(k)
        display_html(v)
        time.sleep(sleep)

iter_display(d)


shooting-opponent


,team,g,mp,opp_fg_pct,opp_avg_dist,DUMMY,opp_pct_fga_fg2a,opp_pct_fga_00_03,opp_pct_fga_03_10,opp_pct_fga_10_16,...,opp_fg_pct_03_10,opp_fg_pct_10_16,opp_fg_pct_16_xx,opp_fg_pct_fg3a,opp_pct_ast_fg2,opp_pct_ast_fg3,opp_pct_fga_dunk,opp_fg_dunk,opp_pct_fg3a_corner,opp_fg3_pct_corner
0,Atlanta Hawks,18,4320,.458,13.6,,.642,.273,.161,.138,...,.419,.384,.404,.351,.494,.792,.052,78,.194,.404
1,Boston Celtics,5,1200,.496,14.3,,.600,.248,.168,.121,...,.394,.471,.667,.426,.500,.708,.066,24,.172,.345
2,Brooklyn Nets,12,2905,.439,14.5,,.607,.239,.162,.127,...,.428,.341,.482,.322,.417,.681,.048,47,.222,.366
3,Dallas Mavericks,7,1680,.488,15.6,,.550,.223,.134,.113,...,.533,.413,.556,.373,.425,.872,.064,33,.274,.391
4,Denver Nuggets,10,2450,.491,15.2,,.594,.202,.174,.121,...,.520,.481,.376,.417,.486,.682,.039,30,.228,.383
5,Los Angeles Clippers,19,4560,.461,15.4,,.577,.200,.189,.113,...,.412,.459,.382,.391,.453,.646,.043,66,.178,.393
6,Los Angeles Lakers,6,1440,.458,14.8,,.603,.244,.118,.157,...,.362,.429,.415,.359,.548,.871,.053,24,.256,.480
7,Memphis Grizzlies,5,1200,.488,15.1,,.521,.215,.215,.069,...,.462,.333,.556,.406,.449,.690,.067,25,.198,.439
8,Miami Heat,4,985,.468,15.1,,.571,.259,.127,.127,...,.417,.500,.500,.327,.492,.868,.066,24,.265,.372
9,Milwaukee Bucks,23,5570,.454,15.6,,.608,.178,.169,.163,...,.407,.538,.416,.352,.440,.729,.040,71,.153,.400


## Series Stats

In [198]:
def get_playoffs_series_game_summaries(series_url='playoffs/2021-nba-finals-bucks-vs-suns.html'):
    source = get_url(f'https://www.basketball-reference.com/{series_url}')
    source = BeautifulSoup(strip_html_comment(source))
    selector = '#div_other_scores > .game_summaries > .game_summary'
    divs = source.select(selector)
    df = []
    for div in divs:
        teams_div,stats_div = div.select('table')
        date,visitor,home = teams_div.select('tr')
        v,pts_v,box_scores = visitor.select('td')
        h,pts_h,_ = home.select('td')

        pts_div,trb_div = stats_div.select('tr')
        _,pts_leader,pts_l = pts_div.select('td')
        _,trb_leader,trb_l = trb_div.select('td')

        game,date = date.text.split(',')
        pts_leader_player,pts_leader_team = pts_leader.text.split('-')
        trb_leader_player,trb_leader_team = trb_leader.text.split('-')
        box_scores = re.findall(r"/boxscores/(.+).html",box_scores.a['href'])[0] if box_scores.a else ""
        d = {
            'game':game,
            'date':date,
            'visitor':v.text,
            'home':h.text,
            'pts_v':pts_v.text,
            'pts_h':pts_h.text,
            'pts_leader_player':pts_leader_player,
            'pts_leader_team':pts_leader_team,
            'pts_l': pts_l.text,
            'trb_leader_player': trb_leader_player,
            'trb_leader_team':trb_leader_team,
            'trb_l': trb_l.text,
            'box_scores':box_scores
        }
        df.append(d)
    return pd.DataFrame(df)


df = get_playoffs_series_game_summaries()
df

,game,date,visitor,home,pts_v,pts_h,pts_leader_player,pts_leader_team,pts_l,trb_leader_player,trb_leader_team,trb_l,box_scores
0,Game 1,Jul 6,Milwaukee Bucks,Phoenix Suns,105,118,C. Paul,PHO,32,D. Ayton,PHO,19,202107060PHO
1,Game 2,Jul 8,Milwaukee Bucks,Phoenix Suns,108,118,G. Antetokounmpo,MIL,42,G. Antetokounmpo,MIL,12,202107080PHO
2,Game 3,Jul 11,Phoenix Suns,Milwaukee Bucks,100,120,G. Antetokounmpo,MIL,41,G. Antetokounmpo,MIL,13,202107110MIL
3,Game 4,Jul 14,Phoenix Suns,Milwaukee Bucks,103,109,D. Booker,PHO,42,D. Ayton,PHO,17,202107140MIL
4,Game 5,Jul 17,Milwaukee Bucks,Phoenix Suns,123,119,D. Booker,PHO,40,D. Ayton,PHO,10,202107170PHO
5,Game 6,Jul 20,Phoenix Suns,Milwaukee Bucks,98,105,G. Antetokounmpo,MIL,50,G. Antetokounmpo,MIL,14,202107200MIL


In [206]:
def get_data_stats(source,selector,select='td'):
    divs = source.select(selector)
    data_stats = pd.DataFrame([{d['data-stat']:d.text for d in div_team.select(select)} for div_team in divs])
    return data_stats

series_url = 'playoffs/2021-nba-finals-bucks-vs-suns.html'
source = get_url(f'https://www.basketball-reference.com/{series_url}')
source = BeautifulSoup(strip_html_comment(source))
selector = '#div_other_scores > .game_summaries > .game_summary'
divs = source.select(selector)


In [207]:
get_data_stats(source,'#four_factors > tbody > tr','td,th')

,team_id,pace,efg_pct,tov_pct,orb_pct,ft_rate,off_rtg,pts
0,MIL (4-2),95.5,.529,9.8,29.2,.190,117.0,111.7
1,PHO (2-4),95.5,.555,11.7,17.4,.176,114.5,109.3


In [208]:
get_data_stats(source,'#MIL > tbody > tr','td,th')

,ranker,player,age,g,mp,fg,fga,fg3,fg3a,ft,...,pts,fg_pct,fg3_pct,ft_pct,mp_per_g,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g
0,1,Giannis Antetokounmpo,26,6,239,76,123,3,15,56,...,211,.618,.200,.659,39.8,35.2,13.2,5.0,1.2,1.8
1,2,Khris Middleton,29,6,255,56,125,16,45,16,...,144,.448,.356,.889,42.5,24.0,6.3,5.3,1.5,0.0
2,3,Jrue Holiday,30,6,250,39,108,11,35,11,...,100,.361,.314,.917,41.7,16.7,6.2,9.3,2.2,0.7
3,4,Brook Lopez,32,6,147,28,58,5,21,8,...,69,.483,.238,.800,24.5,11.5,5.3,0.2,0.7,0.7
4,5,Pat Connaughton,28,6,180,19,40,15,34,2,...,55,.475,.441,.500,30.0,9.2,5.8,1.2,0.2,0.0
5,6,Bobby Portis,25,6,100,17,40,7,16,5,...,46,.425,.438,1.000,16.7,7.7,4.0,0.2,0.7,0.3
6,7,P.J. Tucker,35,6,188,10,20,4,8,0,...,24,.500,.500,,31.3,4.0,3.8,1.2,1.0,0.0
7,8,Bryn Forbes,27,3,22,3,10,3,9,0,...,9,.300,.333,,7.3,3.0,0.3,0.0,0.3,0.0
8,9,Jeff Teague,32,6,56,2,10,1,3,4,...,9,.200,.333,.667,9.3,1.5,0.8,0.7,0.5,0.0
9,10,Jordan Nwora,22,1,1,1,1,1,1,0,...,3,1.000,1.000,,1.0,3.0,1.0,0.0,0.0,0.0


In [209]:
get_data_stats(source,'#MILadvanced > tbody > tr','td,th')

,ranker,player,age,g,gs,mp,ts_pct,efg_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg,game_score
0,1,Giannis Antetokounmpo,26,6,6,239,.658,.630,10.8,27.1,18.5,22.4,1.5,4.0,8.0,31.6,135,111,31.9
1,2,Khris Middleton,29,6,6,255,.542,.512,0.8,17.3,8.5,19.2,1.8,0.0,12.5,25.9,108,115,16.2
2,3,Jrue Holiday,30,6,6,250,.441,.412,2.8,13.9,8.2,31.2,2.6,1.4,10.3,21.9,107,113,15.2
3,4,Brook Lopez,32,6,6,147,.553,.526,7.7,17.3,12.3,1.3,1.4,2.6,8.8,20.2,111,115,8.2
4,5,Pat Connaughton,28,6,0,180,.659,.663,5.6,17.0,11.0,5.1,0.3,0.0,10.7,11.2,127,120,7.3
5,6,Bobby Portis,25,6,0,100,.545,.513,11.1,16.3,13.7,1.5,2.1,1.7,2.3,18.8,123,110,6.3
6,7,P.J. Tucker,35,6,6,188,.600,.600,6.5,6.8,6.7,4.7,1.6,0.0,13.0,5.3,130,117,4.2
7,8,Bryn Forbes,27,3,0,22,.450,.450,0.0,5.0,2.4,0.0,2.3,0.0,9.1,22.1,83,120,0.9
8,9,Jeff Teague,32,6,0,56,.356,.250,4.3,6.5,5.2,8.4,2.8,0.0,19.2,12.3,81,117,1.1
9,10,Jordan Nwora,22,1,0,1,1.500,1.500,0.0,92.3,44.5,0.0,0.0,0.0,0.0,33.4,300,98,3.0
